# Page Template
This notebook contains commands and procedures necessary for all new pages.

v1.2.0

## Global Variables/Commands

In [1]:
# Imports
import templates

In [2]:
# Variables
#target_domain

### Action Template

In [3]:
# Action Template

#force_run = False

#cmd_cache = ''
#cmd_syntax = ''
#cmd_os = ''

#output = templates.cache_command_output(cmd_cache, force_run, cmd_run)

#print(output)

## OSINT Recon

In [ ]:
# Whois
#TODO Use requests to query whois site: https://whois.arin.net/ui/
#TODO Add info recieved into 'loot' directory or page

### Organizational Data
- Organizational Structure
  - Subordinate, senior and lateral organizations
  - Subsidaries
  - Public Presence
  - Organizational Information
- Organizational IT Data
  - Repository commit history
  - Email addresses or email address naming convention
  - Files containing sensitive data and/or metadata - version numbers, applications, process/procedures, etc.
- Data in External Sources
  - Financials that indicate IT spending (security software purchases, etc.)
  - Personnel on social media
  - Customer technologies in resumes, newsgroups, LinkedIn skills, etc.

**Add to Loot**
- Company mission
- Current events related to the company - acquisitions, conferences, etc.
- Points of contact
- Blogs (company and employee's personal)
- Backend infrastructure
- Financial and business information
- Job descriptions for IT positions
- Vendor names

### Github/Pastebin Scraping
- [Truffle Hog](https://github.com/trufflesecurity/trufflehog)
- [Git Harvester](https://github.com/metac0rtex/GitHarvester)
- [TrashPanda - OSINT Bot](https://got-hacked.wtf/)